# Hearthstone Deck Recommendation
### Objetivo: Construir un mazo ganador para una partida de Hearthstone.

## Librerias

In [1]:
import json
import time
import keras
import pickle
import numpy as np
import pandas as pd
import random as rd
from collections import Counter

rd.seed(3633)

## Base de datos
La base de datos se obtuvo de la plataforma HearthPwn (https://www.hearthpwn.com/decks), esta contiene información sobre mazos creados por jugadores de Hearthstone. Los datos incluyen la clase del mazo, sus 30 cartas, el coste de creación, la puntuación, entre otros, pero dentro de las columnas más importantes está el rating que asignaron múltiples jugadores en la plataforma.

In [2]:
df = pd.read_csv("data/data.csv")
df.head()

,craft_cost,date,deck_archetype,deck_class,deck_format,deck_id,deck_set,deck_type,rating,title,...,card_20,card_21,card_22,card_23,card_24,card_25,card_26,card_27,card_28,card_29
0,9740,2016-02-19,Unknown,Priest,W,433004,Explorers,Tavern Brawl,1,Reno Priest,...,374,2280,2511,2555,2566,2582,2683,2736,2568,2883
1,9840,2016-02-19,Unknown,Warrior,W,433003,Explorers,Ranked Deck,1,RoosterWarrior,...,1781,1781,2021,2021,2064,2064,2078,2510,2729,2736
2,2600,2016-02-19,Unknown,Mage,W,433002,Explorers,Theorycraft,1,Annoying,...,1793,1801,1801,2037,2037,2064,2064,2078,38710,38710
3,15600,2016-02-19,Unknown,Warrior,W,433001,Explorers,NaN,0,Standart pay to win warrior,...,1657,1721,2018,2296,2262,336,2729,2729,2736,2760
4,7700,2016-02-19,Unknown,Paladin,W,432997,Explorers,Ranked Deck,1,Palamix,...,2027,2029,2029,2064,2078,374,2717,2717,2889,2889


In [3]:
df.describe()

,craft_cost,deck_id,rating,card_0,card_1,card_2,card_3,card_4,card_5,card_6,...,card_20,card_21,card_22,card_23,card_24,card_25,card_26,card_27,card_28,card_29
count,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,...,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000
mean,5645.360218,394152.425798,2.687230,166.555443,215.682814,328.068948,388.254399,465.439497,530.422084,618.649576,...,6676.817657,7879.285156,9310.508977,10780.480210,12401.890674,13709.898080,15068.109406,15955.823881,19239.277147,20537.999847
std,3927.986295,222605.617140,22.117751,515.546751,549.163776,609.109069,629.218897,710.534945,813.475129,987.250288,...,12610.593768,13750.165455,14885.066464,15862.422929,16731.209629,17295.441879,17766.351821,18039.395857,18567.676290,18686.462303
min,0.000000,18.000000,0.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
25%,2720.000000,216721.500000,1.000000,64.000000,75.000000,189.000000,237.000000,279.000000,304.000000,401.000000,...,1158.000000,1363.000000,1659.000000,1783.000000,1794.000000,1913.000000,2010.000000,2037.000000,2078.000000,2095.000000
50%,5000.000000,406046.500000,1.000000,138.000000,180.000000,285.000000,315.000000,415.000000,475.000000,559.000000,...,1940.000000,2029.000000,2061.000000,2078.000000,2275.000000,2488.000000,2577.000000,2682.000000,2901.000000,3015.000000
75%,7740.000000,590820.500000,1.000000,238.000000,285.000000,421.000000,476.000000,605.000000,643.000000,763.000000,...,2610.000000,2736.000000,2890.000000,2958.000000,38391.000000,38526.000000,38727.000000,38833.000000,38918.000000,39034.000000
max,48000.000000,749548.000000,4016.000000,41409.000000,41409.000000,41609.000000,41409.000000,41409.000000,41609.000000,41609.000000,...,41609.000000,41609.000000,41609.000000,41609.000000,41609.000000,41609.000000,41841.000000,42146.000000,42146.000000,42146.000000


In [4]:
df['deck_class'].value_counts()

deck_class
Mage       45306
Priest     44307
Paladin    42266
Warlock    38022
Druid      37891
Shaman     36457
Warrior    35944
Rogue      34794
Hunter     31245
Name: count, dtype: int64

La clase de mazo con más entradas es "Mage". Para la implementación del sistema de recomendación utilizaremos solo mazos mago.

In [5]:
df = df[df['deck_class'] == 'Mage']

## Cartas y mazos
Definimos el conjunto *cards* con las *N* cartas más repetidas en los mazos mago y *D* como el tamaño del mazo para armar.

In [6]:
cards_cols = df.iloc[:, 11:41] # columnas con las cartas en la base de datos
all_cards = (cards_cols.values.ravel()).tolist() # lista con las cartas
counter = Counter(all_cards) # contador de las cartas

N = len(counter) # tamaño conjunto cartas para elegir
cards =  [item[0] for item in counter.most_common(N)] # conjunto con las N cartas más repetidas

D = 30 # tamaño mazo Hearthstone

El mazo lo definimos como un vector binario *deck* con *N* entradas y exactamente *D* iguales a 1. Este vector nos indica con la i-esima entrada si está o no (1 o 0) en el mazo la i-esima carta de *cards*. Definimos la función *binary_to_cards* que transforma el vector *deck* en una lista con la enumeración de las cartas del mazo. A continación se presenta un ejemplo de mazos.

In [7]:
def binary_to_cards(deck, cards):
    return [int(x) for x in deck*cards if x != 0]

In [8]:
deck = np.zeros(N, dtype=int) # definir mazo jugador
deck[np.random.choice(N, D, replace=False)] = 1 # generar mazo aleatorio jugador
print(f'Vector mazo: {deck}')
print(f'Cartas mazo: {binary_to_cards(deck, cards)}')

Vector mazo: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1
 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0

## Transiciones y acciones
El proceso para mejorar el mazo *deck* consta de aplicar múltiples veces la función de transición *transition* determinada por una acción. Una acción *action* es una tupla con las cartas del reemplazo que modifica exactamente una carta del mazo por una carta que actualmente no está incluida. A continuación, un ejemplo del uso de la función *transition*, en donde se elige de forma aleatoria la acción.

In [9]:
def transition(deck, action):
    deck[action[0]] = 0 # eliminamos la carta antigua del mazo
    deck[action[1]] = 1 # agregamos la carta nueva al mazo
    return deck

In [10]:
print(f'Mazo antiguo: {deck}')
print(f'Cartas mazo antiguo: {binary_to_cards(deck, cards)}')

old_card = np.random.choice(np.where(deck == 1)[0]) # elegimos aleatoriamente una carta que está en el mazo
new_card = np.random.choice(np.where(deck == 0)[0]) # elegimos aleatoriamente una carta que no está en el mazo
action = (old_card, new_card) # definimos la acción

print()
print(f'Acción: ({action[0]},{action[1]})')
print()

new_deck = transition(deck, action)
print(f'Mazo nuevo: {new_deck}')
print(f'Cartas mazo nuevo: {binary_to_cards(new_deck, cards)}')

Mazo antiguo: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1
 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 

## Win Rate
Para elegir las acciones que nos lleven a encontrar un mazo ganador usaremos el *win_rate* de este mazo. Entonces, para elegir la próxima acción buscaremos cual es la que maximiza el *win_rate* del mazo.

Para hacer una función predictora del *win_rate* entrenamos distintos modelos de machine learning, esto se realizó en el notebook *win_rate_prediction.ipynb*. A continuación, se importan estos modelos ya entrenados.

In [11]:
model = keras.models.load_model("win_rate_NN.keras") # importación modelo red neuronal
model_v1 = keras.models.load_model("win_rate_NNv1.keras")
model_v2 = keras.models.load_model("win_rate_NNv2.keras")
model_v3 = keras.models.load_model("win_rate_NNv3.keras")
#model.summary()

with open("encoder", "rb") as f:
    encoder = pickle.load(f)
with open("win_rate_RF.pkl", "rb") as f:
    rf = pickle.load(f) # importación modelo random forest

Ahora, definimos la función *win_rate* que recibe el mazo en el formato enumerado de cartas y dependiendo del modelo elegido entrega el valor del win rate.

In [12]:
def win_rate(deck, encoder = encoder, model = model):
  mazo_gen_num = pd.DataFrame([{f"card_{i}": (deck[i] if i>-1 else 5645) for i in range(-1,30)}]).rename(columns={"card_-1":"craft_cost"})
  hero = "Mage"
  mazo_gen_cat = pd.DataFrame([{"hero":hero}])
  encoded_categorical = encoder.transform(mazo_gen_cat)
  encoded_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out())
  final_df = pd.concat([mazo_gen_num, encoded_df], axis=1).rename(columns={"craft_cost":"dust"})
  if isinstance(model,keras.src.models.sequential.Sequential):
    results = model.predict(final_df,verbose=None)
  else:
    results = model.predict(final_df)
  if isinstance(results[0],np.ndarray):
    return results[0][0]
  else:
    return results[0]

Finalmente, imprimimos el win rate aproximado para el mazo *deck_p* usando los dos modelos. Primero en base a la red neuronal entrenada y después mediante un random forest.

In [13]:
print('Neural network:', win_rate(binary_to_cards(deck, cards))) # red neuronal
print('Random forest:', win_rate(binary_to_cards(deck, cards),model=rf)) # random forest
print('Neural network v1:', win_rate(binary_to_cards(deck, cards),model=model_v1)) # red neuronal
print('Neural network v2:', win_rate(binary_to_cards(deck, cards),model=model_v2)) # red neuronal
print('Neural network v3:', win_rate(binary_to_cards(deck, cards),model=model_v3)) # red neuronal


Neural network: 54.92001
Random forest: 50.51133333333337
Neural network v1: 31.473282
Neural network v2: 28.782068
Neural network v3: 37.498074


## Implementación 

Con la función de *win_rate* ya definida podemos decidir cual es la siguiente acción más conveniente a realizar. Vamos a comenzar con un mazo *deck* y realizar acciones hasta que ya no sea conveniente seguir modificando el mazo, estas serán a los más *D*, ya que a lo más cambiamos todo el mazo para encontrar el óptimo. A continuación, definimos la función *next_action* que a partir del mazo actual encuentra la siguiente acción más conveniente a realizar según el *win_rate*, incluyendo la acción de preservar el mazo igual.

In [70]:
def next_action(deck, cards):
    current_win_rate = win_rate(binary_to_cards(deck, cards))
    best_win_rate = current_win_rate
    best_action = None
    
    zeros = [index for index, value in enumerate(deck) if value == 0]
    ones = [index for index, value in enumerate(deck) if value == 1]
     
    for zero in zeros:
        for one in ones: 
            temp_deck = deck.copy()  
            action = (one, zero)
            temp_deck = transition(temp_deck, action)
            temp_win_rate = win_rate(binary_to_cards(temp_deck, cards),model=model_v3)
            if temp_win_rate > best_win_rate:
                best_win_rate = temp_win_rate
                best_action = action
    
    return best_action    

Ahora con la función *deck_reck* podemos hacer efectiva la mejor acción sucesivamente hasta llegar a *D* acciones o haber generado un mazo óptimo.

In [15]:
def deck_rec(deck, cards):
    for i in range(D):
        action = next_action(deck, cards)
        if action == None:
            break
        else:
            deck = transition(deck, action)
    return deck

In [17]:
print(f'Mazo inicial: {binary_to_cards(deck, cards)}')
print(f'Win rate inicial: {win_rate(binary_to_cards(deck, cards))}')
new_deck = deck_rec(deck, cards)
print(f'Mazo final: {binary_to_cards(new_deck, cards)}')
print(f'Win rate final: {win_rate(binary_to_cards(new_deck, cards))}')

Mazo inicial: [172, 2053, 2883, 38859, 436, 2539, 1186, 1155, 1794, 40701, 39489, 41, 700, 921, 712, 1796, 1687, 2072, 2085, 453, 2503, 39225, 736, 2533, 1370, 2043, 39313, 2595, 38569, 2585]
Win rate inicial: 54.92000961303711


KeyboardInterrupt: 

## Comparación de resultados
### Distintos Métodos
Vamos a comparar nuestra implementación con los siguientes métodos.

1. Random:
Entrega de forma aleatoria un mazo de cartas de la base de datos.

In [30]:
def random_deck():
  row = df.sample(n=1)
  id = row['deck_id'].tolist()[0]
  cards_cols = row.iloc[:, 11:41]
  cards_list = (cards_cols.values.ravel()).tolist() 
  return id, cards_list

2. Most popular: 
Retorna el mazo con el mejor rating de la base de datos.

In [19]:
def most_popular_deck():
  row = df[df['rating'] == df['rating'].max()]
  cards_cols = row.iloc[:, 11:41]
  cards_list = (cards_cols.values.ravel()).tolist() 
  return cards_list

3. Best Similar:
Recibe un mazo y retorna un mazo similar de la base de datos con mejor rating. 

Para definir la similitud, se implementó una función *difference* que calcula la cantidad de cartas diferentes entre los mazos. De esta forma, el modelo también recibe como parámetro un *delta* que define la máxima posible diferencia entre el mazo retornado y el original.

In [20]:
def difference(deck_1, deck_2):
  common = len(np.intersect1d(deck_1, deck_2))
  diff = len(deck_1) - common
  return diff

def best_similar_deck(my_deck, delta):
  best_decks = df[df['rating'] > 2]
  best_deck_id = -1
  best_deck_rating = -1
  for index, row in best_decks.iterrows():
    new_deck_id = row['deck_id']
    new_deck_rating = row['rating']
    deck_2 = df[df['deck_id'] == new_deck_id].iloc[:, 11:41].values.flatten()
    if difference(my_deck, deck_2) <= delta:
      if new_deck_rating >= best_deck_rating:
        best_deck_id = new_deck_id
        best_deck_rating = new_deck_rating
  if best_deck_id>=0:
    row = df[df['deck_id'] == best_deck_id]
    cards_cols = row.iloc[:, 11:41]
    cards_list = (cards_cols.values.ravel()).tolist() 
    return cards_list
  else:
    return my_deck

Entonces, a continuación vamos a seleccionar de forma aleatoria un mazo de la base de datos y vamos a comparar los distintos sistemas recomendadores usando el *win_rate*.

In [58]:
original_id, original_deck = random_deck() # generamos mazo aleatorio
print(f'Mazo inicial: {original_deck}')
print(f'Win rate inicial: {win_rate(original_deck,model=model_v3)}')

Mazo inicial: [77, 113, 113, 315, 315, 395, 395, 531, 555, 555, 581, 662, 662, 1004, 1186, 1721, 1793, 1793, 1808, 1808, 1927, 1927, 1928, 1928, 2044, 2044, 2078, 374, 38710, 38710]
Win rate inicial: 27.771785736083984


In [64]:
id_random, deck_random = random_deck() # generamos recomendación aleatoria
print(f'Recomendación random: {deck_random}')
print(f'Win rate random: {win_rate(deck_random,model=model_v3)}')

Recomendación random: [77, 113, 192, 195, 195, 286, 315, 315, 366, 366, 621, 621, 662, 662, 748, 748, 1004, 1080, 1087, 1737, 1737, 1783, 1783, 1801, 1927, 1927, 2037, 2057, 38710, 38710]
Win rate random: 29.248106002807617


In [65]:
deck_most_popular = most_popular_deck() # generamos recomendación más popular
print(f'Recomendación más popular: {deck_most_popular}')
print(f'Win rate más popular: {win_rate(deck_most_popular,model=model_v3)}')

Recomendación más popular: [77, 315, 315, 405, 405, 555, 564, 564, 614, 635, 662, 662, 825, 825, 1004, 38547, 38547, 38725, 38725, 38857, 38859, 38859, 38863, 38863, 38868, 38868, 38900, 38900, 39715, 39715]
Win rate más popular: 34.774356842041016


In [66]:
id_random, deck_random = random_deck()
deck_best_similar = best_similar_deck(deck_random, 25) # generamos recomendación mejor similar
print(f'Recomendación mejor similar: {deck_best_similar}')
print(f'Win rate mejor similar: {win_rate(deck_best_similar,model=model_v3)}')

Recomendación mejor similar: [77, 315, 315, 405, 405, 555, 564, 564, 614, 635, 662, 662, 825, 825, 1004, 38547, 38547, 38725, 38725, 38857, 38859, 38859, 38863, 38863, 38868, 38868, 38900, 38900, 39715, 39715]
Win rate mejor similar: 34.774356842041016


In [ ]:
N=40 # Este N se utiliza solo en esta celda, para el resto se debe usar el original N=len(counter)
deck = np.zeros(N, dtype=int)
for i in range(N):
    for card in original_deck:
        if card == cards[i]:
            deck[i] = 1
zeros = [index for index, value in enumerate(deck) if value == 0]
ones = [index for index, value in enumerate(deck) if value == 1]
new_ones = rd.sample(zeros,D-len(ones))
for i in new_ones:
    deck[i] = 1
cards_subset = cards[:N]
deck_our_rec = deck_rec(deck,cards_subset)
print(f'Nuestra recomendación : {binary_to_cards(deck_our_rec, cards_subset)}')
print(f'Win rate nuestra recomendación: {win_rate(binary_to_cards(deck_our_rec, cards_subset),model=model_v3)}')

Nuestra recomendación : [662, 315, 555, 1004, 77, 825, 1783, 395, 1084, 38710, 172, 2874, 1659, 1927, 1793, 113, 2875, 1928, 430, 1087, 2050, 2078, 581, 1940, 39715, 374, 2070, 1721, 906, 2883]
Win rate nuestra recomendación: 31.558528900146484


In [137]:
def next_action_restricted(deck, cards, fixed):
    current_win_rate = win_rate(binary_to_cards(deck, cards),model=model_v3)
    best_win_rate = current_win_rate
    best_action = None
    zeros = [index for index, value in enumerate(deck) if ((value == 0) and (index not in fixed))]
    ones = [index for index, value in enumerate(deck) if ((value == 1) and (index not in fixed))]
    for zero in zeros:
        for one in ones: 
            temp_deck = deck.copy()  
            action = (one, zero)
            temp_deck = transition(temp_deck, action)
            temp_win_rate = win_rate(binary_to_cards(temp_deck, cards),model=model_v3)
            if temp_win_rate > best_win_rate:
                best_win_rate = temp_win_rate
                best_action = action
    
    return best_action   

def Qdeck_wtemplate(my_deck, delta, gamma,N=40):
    cards_list = best_similar_deck(my_deck, delta)
    deck = np.zeros(N, dtype=int)
    for i in range(N):
        for card in cards_list:
            if card == cards[i]:
                deck[i] = 1
    zeros = [index for index, value in enumerate(deck) if value == 0]
    ones = [index for index, value in enumerate(deck) if value == 1]
    new_ones = rd.sample(zeros,D-len(ones))
    fixed_c = rd.sample(np.arange(N).tolist(),gamma)
    for i in new_ones:
        deck[i] = 1
    zeros = [i for i in zeros if i not in fixed_c]
    ones = [i for i in ones if i not in fixed_c]
    cards_subset = cards[:N]
    for i in range(D-len(fixed_c)):
        action = next_action(deck, cards_subset)
        if action == None:
            break
        else:
            deck = transition(deck, action)
    return deck



In [106]:
cards_subset = cards[:40]

In [ ]:
deck_qdecktemp = Qdeck_wtemplate(original_deck, 20, 20)
print(f'Recomendación qdeck con templates: {binary_to_cards(deck_qdecktemp, cards_subset)}')
print(f'Win rate qdeck con templates: {win_rate(binary_to_cards(deck_qdecktemp, cards_subset),model=model_v3)}')

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 0 1 1 1 1 1 1 0 1 0 0 1 0 0
 1 1 1]
Recomendación qdeck con templates: [662, 315, 555, 1004, 405, 614, 77, 825, 564, 1783, 395, 457, 2275, 1084, 587, 195, 172, 621, 138, 2064, 1801, 1927, 1793, 2572, 2262, 113, 1928, 1087, 2050, 2078]
Win rate qdeck con templates: 51.95368576049805


Observamos que de los sistemas recomendadores anteriores el que entrega el mazo con un mejor *win rate* es el nuestro.
### Distintos mazos
Ahora, vamos a implementar lo anterior para múltiples mazos aleatorios y volver a comparar. En particular hacemos una prueba para 20 mazos iniciales, con el fin de comparar la mejora relacionada al mazo final obtenido con cada método.

In [109]:
simulations = {"initial":[], "random":[],"MP":[],"BS":[],"DeckReck":[],"templateDeckReck":[]}
N = 50
for i in range(20):
    initial_deck = random_deck()[1]
    simulations["initial"].append(win_rate(initial_deck,model=model_v3))
    # random
    simulations["random"].append(win_rate(random_deck()[1],model=model_v3))
    # most popular 
    simulations["MP"].append(win_rate(most_popular_deck(),model=model_v3))
    # best similar
    simulations["BS"].append(win_rate(best_similar_deck(initial_deck, 25),model=model_v3))
    # Qdeck con template
    deck_qdecktemp = Qdeck_wtemplate(original_deck, 20, 20,N = 50)
    simulations["templateDeckReck"].append(win_rate(binary_to_cards(deck_qdecktemp, cards_subset),model=model_v3))
    deck = np.zeros(N, dtype=int)
    for i in range(N):
        for card in initial_deck:
            if card == cards[i]:
                deck[i] = 1
    zeros = [index for index, value in enumerate(deck) if value == 0]
    ones = [index for index, value in enumerate(deck) if value == 1]
    new_ones = rd.sample(zeros,D-len(ones))
    for i in new_ones:
        deck[i] = 1
    cards_subset = cards[:N]
    deck_our_rec = deck_rec(deck,cards_subset)
    simulations["DeckReck"].append(win_rate(binary_to_cards(deck_our_rec, cards_subset),model=model_v3))

[1 1 0 1 1 1 0 1 0 1 0 0 0 1 0 0 1 1 0 0 1 0 0 1 1 1 1 1 0 1 0 1 1 0 0 1 1
 0 1 1 0 1 1 1 0 0 1 1 1 1]
[1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 0 1 1 0 1 0 0 0 1 1 1 1 1 0 1 0 1 1 0 0 0 0
 0 0 1 0 0 1 1 0 0 1 1 1 1]
[1 1 0 1 1 1 0 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 0 0 0 1 0 0 0 0 0
 1 1 1 0 1 1 1 1 0 0 1 1 0]
[1 1 0 1 1 1 0 1 0 1 0 0 0 1 1 0 1 1 0 0 0 1 0 1 1 1 1 1 0 0 1 1 0 1 0 1 0
 0 1 1 1 1 0 1 0 1 1 0 1 1]
[1 1 0 1 1 1 0 1 0 1 1 0 1 0 1 1 1 1 0 0 0 0 1 1 1 0 1 1 0 0 1 1 0 1 0 1 1
 0 0 1 1 0 1 1 1 0 0 0 1 1]
[1 1 0 1 1 1 1 1 0 1 0 1 0 0 0 1 1 1 0 1 0 0 0 1 1 1 1 1 0 1 1 1 0 0 1 1 0
 1 1 1 0 0 1 1 0 0 0 1 1 0]
[1 1 0 1 1 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 1 1 1 1 0 1 0 0 1 0 0
 0 1 1 1 1 1 1 0 1 0 0 1 0]
[1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 0 1 1 0 0 1 0 0 0 1 0 1 1 0 1 1 1 0 1 1 0 1
 0 1 1 0 0 0 1 1 0 1 1 1 0]
[1 1 1 1 1 1 1 1 0 1 1 0 0 1 0 0 1 1 1 1 0 0 0 0 1 1 1 1 1 0 0 1 1 0 1 0 0
 0 0 1 1 0 0 1 1 0 0 1 1 1]
[1 1 0 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 0 0 0 1 0 1 1 0 1 1 1 0 0 1 1 1 0 1 

In [110]:
# esta celda es utilizada únicamente para un guardado de resultados en un formato cómodo
print(simulations["initial"])
sim_fl = dict()
for key in simulations.keys():
    sim_fl[key] = [float(simulations[key][i]) for i in range(len(simulations[key]))]

[np.float32(32.01451), np.float32(33.158554), np.float32(31.094265), np.float32(48.008743), np.float32(50.608967), np.float32(28.535728), np.float32(30.13098), np.float32(31.557222), np.float32(29.598967), np.float32(51.1679), np.float32(51.501724), np.float32(33.24939), np.float32(48.28563), np.float32(29.520306), np.float32(51.20398), np.float32(29.51111), np.float32(25.686136), np.float32(29.009476), np.float32(29.598967), np.float32(46.818153)]


In [111]:
#guardando resultados
with open('comparacion_modelos.json', 'w') as json_file:
    json.dump(sim_fl, json_file, indent=4)

vemos los resultados para cada intento

In [112]:
comparacion = pd.DataFrame(sim_fl)
comparacion#.describe()

,initial,random,MP,BS,DeckReck,templateDeckReck
0,32.014511,30.314335,34.774357,34.774357,53.447742,29.550455
1,33.158554,31.214071,34.774357,34.774357,26.107454,32.950394
2,31.094265,49.714924,34.774357,34.774357,29.347555,18.166197
3,48.008743,29.598967,34.774357,34.774357,26.904556,36.722961
4,50.608967,51.548786,34.774357,34.774357,52.776047,28.978338
5,28.535728,50.795357,34.774357,50.310116,28.751604,27.928335
6,30.130980,47.174068,34.774357,34.774357,27.814301,27.660873
7,31.557222,50.157482,34.774357,34.774357,31.599485,22.175159
8,29.598967,41.200554,34.774357,34.774357,32.593365,29.068571
9,51.167900,50.440674,34.774357,50.310116,23.167984,25.035269


In [122]:
aumentos = pd.DataFrame({"random":comparacion["random"]/comparacion["initial"],
                         "MP": comparacion["MP"]/comparacion["initial"],
                         "BS": comparacion["BS"]/comparacion["initial"],
                         "DeckReck": comparacion["DeckReck"]/comparacion["initial"],
                         "temp_DeckReck": comparacion["templateDeckReck"]/comparacion["initial"]})
aumentos

,random,MP,BS,DeckReck,temp_DeckReck
0,0.946894,1.086206,1.086206,1.669485,0.923033
1,0.941358,1.048730,1.048730,0.787352,0.993722
2,1.598845,1.118353,1.118353,0.943825,0.584230
3,0.616533,0.724334,0.724334,0.560410,0.764922
4,1.018570,0.687118,0.687118,1.042820,0.572593
5,1.780062,1.218625,1.763057,1.007565,0.978715
6,1.565633,1.154106,1.154106,0.923113,0.918021
7,1.589414,1.101946,1.101946,1.001339,0.702697
8,1.391959,1.174850,1.174850,1.101166,0.982081
9,0.985787,0.679613,0.983236,0.452784,0.489277


el dataframe aumentos representa la mejora porcentual de cada método y sus estadísticas están dadas por el describe a continuación. 

In [114]:
aumentos.describe()

,random,MP,BS,DeckReck
count,20.000000,20.000000,20.000000,20.000000
mean,1.169400,0.997037,1.032403,0.874160
std,0.406891,0.232429,0.283765,0.296398
min,0.575228,0.675208,0.602014,0.408976
25%,0.935811,0.723295,0.723295,0.629676
50%,1.019511,1.094076,1.094076,0.933469
75%,1.586938,1.175633,1.175633,1.045147
max,1.780062,1.353818,1.763057,1.669485


### Distintos parámetros

a continuación probamos variando el parámetro delta del método de Best most similar

In [117]:
simulations_delta = {"initial":[], "25":[],"20":[],"15":[],"10":[]}
for i in range(20):
    initial_deck = random_deck()[1]
    simulations_delta["initial"].append(win_rate(initial_deck,model=model_v3))
    simulations_delta["25"].append(win_rate(best_similar_deck(initial_deck, 25),model=model_v3))
    simulations_delta["20"].append(win_rate(best_similar_deck(initial_deck, 20),model=model_v3))
    simulations_delta["15"].append(win_rate(best_similar_deck(initial_deck, 15),model=model_v3))
    simulations_delta["10"].append(win_rate(best_similar_deck(initial_deck, 10),model=model_v3))
print(simulations["initial"])
sim_delta_fl = dict()
for key in simulations_delta.keys():
    sim_delta_fl[key] = [float(simulations_delta[key][i]) for i in range(len(simulations_delta[key]))]
with open('comparacion_delta.json', 'w') as json_file:
    json.dump(sim_delta_fl, json_file, indent=4)
comparacion_delta = pd.DataFrame(simulations_delta)
comparacion_delta

[np.float32(32.01451), np.float32(33.158554), np.float32(31.094265), np.float32(48.008743), np.float32(50.608967), np.float32(28.535728), np.float32(30.13098), np.float32(31.557222), np.float32(29.598967), np.float32(51.1679), np.float32(51.501724), np.float32(33.24939), np.float32(48.28563), np.float32(29.520306), np.float32(51.20398), np.float32(29.51111), np.float32(25.686136), np.float32(29.009476), np.float32(29.598967), np.float32(46.818153)]


,initial,25,20,15,10
0,34.818684,36.115280,30.716143,30.716143,46.533451
1,48.418850,49.659348,30.716143,46.533451,48.418850
2,30.314335,34.774357,30.187626,30.314335,30.314335
3,28.111349,34.774357,36.115280,32.013607,28.111349
4,50.842415,49.419254,50.842415,50.842415,50.842415
5,27.866520,34.774357,36.115280,29.248104,27.866520
6,33.790287,34.774357,36.115280,34.667816,33.790287
7,48.408760,50.310116,51.291206,48.408760,48.408760
8,27.555473,34.774357,28.185167,28.185167,27.555473
9,28.535976,34.774357,28.185167,29.248104,28.535976


In [118]:
aumentos_delta = pd.DataFrame({"25":comparacion_delta["25"]/comparacion_delta["initial"],
                         "20": comparacion_delta["20"]/comparacion_delta["initial"],
                         "15": comparacion_delta["15"]/comparacion_delta["initial"],
                         "10": comparacion_delta["10"]/comparacion_delta["initial"]})
aumentos_delta.describe()

,25,20,15,10
count,20.000000,20.000000,20.000000,20.000000
mean,1.047218,0.960375,1.011494,1.017709
std,0.188094,0.205072,0.061767,0.075128
min,0.677903,0.563533,0.882174,1.000000
25%,0.975251,0.890520,1.000000,1.000000
50%,1.070709,0.997910,1.000000,1.000000
75%,1.206867,1.043194,1.025209,1.000000
max,1.261976,1.296010,1.164743,1.336451


A partir de aquí vemos comparaciones en tiempo y resultados de DeckRec para distintas cantidades de cartas a usar (N)

In [121]:
simulations_N = {"initial":[], 40:[],55:[],70:[]}
times_N = {"initial":[], 40:[],55:[],70:[]}
Ns = [40,55,70]
for i in range(15):
    print(i)
    initial_deck = random_deck()[1]
    simulations_N["initial"].append(win_rate(initial_deck,model=model_v3))
    for N in Ns:
        deck = np.zeros(N, dtype=int)
        for j in range(N):
            for card in initial_deck:
                if card == cards[j]:
                    deck[j] = 1
        zeros = [index for index, value in enumerate(deck) if value == 0]
        ones = [index for index, value in enumerate(deck) if value == 1]
        new_ones = rd.sample(zeros,D-len(ones))
        for j in new_ones:
            deck[j] = 1
        cards_subset = cards[:N]
        start = time.time()
        deck_our_rec = deck_rec(deck,cards_subset)
        end = time.time()
        times_N[N].append(end-start)
        simulations_N[N].append(win_rate(binary_to_cards(deck_our_rec, cards_subset),model=model_v3))
sim_N_fl = dict()
for key in simulations_N.keys():
    sim_N_fl[f'{key}'] = [float(simulations_N[key][i]) for i in range(len(simulations_N[key]))]
with open('comparacion_N.json', 'w') as json_file:
    json.dump(sim_N_fl, json_file, indent=4)
comparacion_N = pd.DataFrame(simulations_N)
display(comparacion_N)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


,initial,40,55,70
0,50.877644,52.420666,26.954014,52.778412
1,31.382158,51.885555,28.327192,21.721256
2,30.561817,51.769604,31.503832,24.849821
3,30.561817,51.563633,28.674547,24.891111
4,38.132912,47.826035,20.366564,21.927982
5,47.949020,22.039642,35.041611,25.552048
6,27.393749,22.449018,21.737896,19.816553
7,29.072124,48.774933,28.240038,37.412247
8,28.110918,27.836966,28.006954,20.828495
9,33.788162,53.808456,33.751823,28.948643


In [ ]:
for key in times_N.keys():
    if key != initial
    print(key)
    print(len(times_N[key]))

initial
0
40
15
55
15
70
15


In [129]:
sim_T_fl = dict()
for key in times_N.keys():
    if key != "initial":
        sim_T_fl[f'{key}'] = [float(times_N[key][i]) for i in range(len(times_N[key]))]
with open('comparacion_T.json', 'w') as json_file:
    json.dump(sim_T_fl, json_file, indent=4)
comparacion_T = pd.DataFrame(sim_T_fl)
display(comparacion_T)

,40,55,70
0,25.299482,31.139656,101.000869
1,26.535413,31.697711,50.698926
2,12.780151,32.801363,52.628749
3,35.398657,49.565457,80.008800
4,14.638157,46.834515,80.416688
5,19.805651,51.347312,78.229944
6,12.378294,43.599172,80.270205
7,19.846969,50.026190,79.765366
8,19.977685,50.855169,73.792069
9,82.239077,49.571779,79.463626


In [131]:
comparacion_T.describe()

,40,55,70
count,15.000000,15.000000,15.000000
mean,23.633137,40.749246,65.868857
std,17.732336,10.676473,18.433070
min,11.554350,28.731303,46.186607
25%,12.579222,30.093526,46.507324
50%,19.846969,43.599172,73.792069
75%,25.917447,49.798985,79.887083
max,82.239077,58.044976,101.000869


In [ ]:
Times = []
Ns = [40,55,70]
initial_deck = random_deck()[1]
for N in Ns:
    deck = np.zeros(N, dtype=int)
    for j in range(N):
        for card in initial_deck:
            if card == cards[j]:
                deck[j] = 1
    zeros = [index for index, value in enumerate(deck) if value == 0]
    ones = [index for index, value in enumerate(deck) if value == 1]
    new_ones = rd.sample(zeros,D-len(ones))
    for j in new_ones:
        deck[j] = 1
    cards_subset = cards[:N]
    start = time.time()
    deck_our_rec = deck_rec(deck,cards_subset)
    end = time.time()
    Times.append(end-start)
print(f'N=40: {Times[0]} segundos\nN=55: {Times[1]} segundos\nN=70: {Times[2]} segundos\n')

N=40: 65.29655432701111 segundos
N=55: 97.52758812904358 segundos
N=70: 287.7180218696594 segundos



In [142]:
simulations_N = {"initial":[], 40:[],55:[],70:[]}
times_N = {"initial":[], 40:[],55:[],70:[]}
Ns = [40,55,70]
for i in range(5):
    print(i)
    initial_deck = random_deck()[1]
    simulations_N["initial"].append(win_rate(initial_deck,model=model_v3))
    for N in Ns:
        start = time.time()
        deck_best_similar = Qdeck_wtemplate(initial_deck, 15, 15, N=N)
        end = time.time()
        times_N[N].append(end-start)
        simulations_N[N].append(win_rate(binary_to_cards(deck_best_similar, cards_subset[:N]),model=model_v3))
sim_N_fl = dict()
for key in simulations_N.keys():
    sim_N_fl[f'{key}'] = [float(simulations_N[key][i]) for i in range(len(simulations_N[key]))]
with open('comparacion_N_mixed.json', 'w') as json_file:
    json.dump(sim_N_fl, json_file, indent=4)
comparacion_N = pd.DataFrame(simulations_N)
display(comparacion_N)

sim_T_fl = dict()
for key in times_N.keys():
    if key != "initial":
        sim_T_fl[f'{key}'] = [float(times_N[key][i]) for i in range(len(times_N[key]))]
with open('comparacion_T_mixed.json', 'w') as json_file:
    json.dump(sim_T_fl, json_file, indent=4)
comparacion_T = pd.DataFrame(sim_T_fl)
display(comparacion_T)

0
1
2
3
4


,initial,40,55,70
0,49.026386,49.021130,25.336514,21.263823
1,30.182905,22.574497,33.669983,30.010733
2,28.907530,53.419788,31.369591,25.293636
3,30.183685,49.386124,30.753443,33.953781
4,30.314335,51.260143,26.068607,30.366371


,40,55,70
0,25.329907,31.028491,49.207912
1,13.296545,31.481771,49.397964
2,13.071202,31.524327,49.612356
3,12.855858,30.324496,48.386706
4,25.241593,30.719541,48.021789


In [144]:
print("WR vs Cantidad de cartas usadas (N):")
display(comparacion_N.describe())
print("tiempos en segundos:")
display(comparacion_T.describe())

WR vs Cantidad de cartas usadas (N):


,initial,40,55,70
count,5.000000,5.000000,5.000000,5.000000
mean,33.722969,45.132336,29.439627,28.177668
std,8.574095,12.730921,3.589781,4.940469
min,28.907530,22.574497,25.336514,21.263823
25%,30.182905,49.021130,26.068607,25.293636
50%,30.183685,49.386124,30.753443,30.010733
75%,30.314335,51.260143,31.369591,30.366371
max,49.026386,53.419788,33.669983,33.953781


tiempos en segundos:


,40,55,70
count,5.000000,5.000000,5.000000
mean,17.959021,31.015725,48.925345
std,6.690246,0.510284,0.685883
min,12.855858,30.324496,48.021789
25%,13.071202,30.719541,48.386706
50%,13.296545,31.028491,49.207912
75%,25.241593,31.481771,49.397964
max,25.329907,31.524327,49.612356
